In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as sci
import matplotlib.pyplot as plt
import json
from astropy.time import Time as astrotime
from anapymods3.general import idx2rowcol

from tdepps.utils import (create_run_dict, make_rate_records, rebin_rate_rec,
                          make_spl_edges, fit_spl_to_hist, arr2str,
                          power_law_flux, dict_map)
from tdepps.grb import GRBLLH, GRBModel, MultiGRBLLH
from tdepps.grb import (SignalFluenceInjector, UniformTimeSampler,
                        TimeDecDependentBGDataInjector)
from tdepps.grb import MultiBGDataInjector, MultiSignalFluenceInjector
from tdepps.grb import GRBLLHAnalysis
from tdepps.grb import SinusFixedConstRateFunction

secinday = 24. * 60. * 60.
rndgen = np.random.RandomState(42439462)
print("Started: ", astrotime.now())

## Setup code

In [ ]:
def setup_data(sample_name):
    names = ["79", "86I", "86II", "86III"]

    print("# Loading experimental data arrays:")
    exp = {}
    p = "/Users/tmenne/git/phd/hese_tdepps/data/proc/offdata"
    for n in names:
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, "{}.npy".format(n))
            exp[n] = np.load(p_)
            print(" - {}: '{}'".format(n, p_))


    print("# Loading Monte Carlo data arrays:")
    mc = {}
    files_ = ["IC79/IC79_corrected_MC.npy",
              "IC86_2011/IC86_corrected_MC.npy",
              "IC86_2012/IC86-2012_corrected_MC.npy",
              "IC86_2012/IC86-2012_corrected_MC.npy"]
    p = "/Users/tmenne/git/phd/hese_tdepps/data/raw"
    for n, f_ in zip(names, files_):
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, f_)
            mc[n] = np.load(p_)
            print(" - {}: '{}'".format(n, p_))

    # We need "timeMJD" as a name, so change the dtype names
    for mci in mc.values():
        if "timeMJD" not in mci.dtype.names:
            idx = mci.dtype.names.index("time")
            mci.dtype.names = (mci.dtype.names[:idx] + ("timeMJD",) +
                               mci.dtype.names[idx + 1:])


    print("# Loading runlists:")
    rundict = {}
    files_ = ["IC79v24.json", "IC86_2011.json",
              "IC86_2012.json", "IC86_2013.json"]
    p = "/Users/tmenne/git/phd/hese_tdepps/data/proc/goodruns"
    for n, f_ in zip(names, files_):
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, f_)
            rundict[n] = json.load(open(p_))
            print(" - {}: '{}'".format(n, p_))


    print("# Load HESE tracks locations and prepare srcs:")
    p = os.path.join("/Users/tmenne/git/phd/hese_tdepps/data/raw/",
                     "public_data_release/All_HESE_Events_4_years_tracks.txt")
    src_t, src_dec, src_ra = np.loadtxt(p, usecols=[1, 2, 3], unpack=True)
    src_ra = np.deg2rad(src_ra)
    src_dec = np.deg2rad(src_dec)

    src_arr_names = ["t", "dt0", "dt1", "ra", "dec", "w_theo"]
    types = len(src_arr_names) * [np.float]
    dtype = [(name, typ) for name, typ in zip(src_arr_names, types)]

    srcs = {}
    for key, rl in rundict.items():
        runs = rl["runs"]
        tmin = np.amin([astrotime(r["good_tstart"]).mjd for r in runs])
        tmax = np.amax([astrotime(r["good_tstop"]).mjd for r in runs])
        t_mask = (src_t >= tmin) & (src_t <= tmax)

        src_ti = src_t[t_mask]
        nsrcs_i = len(src_ti)

        srcs_i = np.empty((nsrcs_i, ), dtype=dtype)

        srcs_i["t"] = src_ti
        # Leave empty for now, time window is set explicitely below
        srcs_i["dt0"] = np.zeros(nsrcs_i, dtype=np.float)
        srcs_i["dt1"] = np.zeros(nsrcs_i, dtype=np.float)
        srcs_i["ra"] = src_ra[t_mask]
        srcs_i["dec"] = src_dec[t_mask]
        srcs_i["w_theo"] = np.ones(nsrcs_i, dtype=np.float)
        srcs[key] = srcs_i

    p = ("/Users/tmenne/git/phd/hese_tdepps/data/proc/" +
         "time_windows/time_windows.txt")
    time_window = np.loadtxt(fname=p)[tw_id]
    print("   + Loaded time window list: '{}'".format(p))
    print("   + Using time window {:2d}: {:.2f}s".format(tw_id,
                                                       np.diff(time_window)[0]))
    for key, srcs_i in sorted(srcs.items()):
        print(" - {}:".format(key))
        srcs_i["dt0"] = np.repeat(time_window[0], repeats=len(srcs_i))
        srcs_i["dt1"] = np.repeat(time_window[1], repeats=len(srcs_i))
        for n in srcs_i.dtype.names:
            print("   + {}: [{}]".format(n, arr2str(srcs_i[n], fmt="{:.2f}")))
            
    # If all selected, concat IC86II & III, because they have the same MC    
    if sample_name == "ALL":
        exp["86II_III"] = np.concatenate((exp["86II"], exp["86III"]))
        _ = exp.pop("86II")
        _ = exp.pop("86III")
        
        mc["86II_III"] = mc["86II"]
        _ = mc.pop("86II")
        _ = mc.pop("86III")
        
        srcs["86II_III"] = np.concatenate((srcs["86II"], srcs["86III"]))
        _ = srcs.pop("86II")
        _ = srcs.pop("86III")
        
        rundict["86II_III"] = {
            "runs": rundict["86II"]["runs"] + rundict["86III"]["runs"]}
        _ = rundict.pop("86II")
        _ = rundict.pop("86III")
        print("All samples selected, combined 86II and 86III tp 86II_III.")
                
    return exp, mc, srcs, rundict

## Prepare

In [ ]:
# Loading data setup
tw_id = 15
sample_name = "ALL"  # One of: 79, 86I, 86II, 86III or ALL
if sample_name not in ["79", "86I", "86II", "86III", "ALL"]:
    raise ValueError("Wrong name for `n` chosen.")

exp, mc, srcs, rundict = setup_data(sample_name)

# Single year aliases for testing
if sample_name is not "ALL":
    exp_ = exp[sample_name]
    MC_ = mc[sample_name]
    srcs_ = srcs[sample_name]
    run_dict_ = create_run_dict(rundict[sample_name]["runs"])
else:
    exp_ = exp
    MC_ = mc
    srcs_ = srcs
    run_dict_ = {key: create_run_dict(rd["runs"]) for key, rd
                 in rundict.items()}

print("\nDone, loaded sample(s) for '{}'.".format(sample_name))

## Single sample

In [ ]:
# Build and fit bg injector
# Finer resolution around the horizon region, where we usually switch the event
# selections from northern to southern samples
hor = np.sin(np.deg2rad(30))
sindec_bins = np.unique(np.concatenate([
                        np.linspace(-1., -hor, 3 + 1),    # south
                        np.linspace(-hor, +hor, 14 + 1),  # horizon
                        np.linspace(+hor, 1., 3 + 1),     # north
                        ]))
rate_rebins = np.linspace(exp_["timeMJD"].min(), exp_["timeMJD"].max(), 12)
# Choose spl_s so that the spline sticks a little more to the data
bg_inj_opts = {"sindec_bins": sindec_bins, "rate_rebins": rate_rebins,
               "spl_s": len(sindec_bins) // 2, "n_scan_bins": 25}
bg_inj = TimeDecDependentBGDataInjector(inj_opts=bg_inj_opts, rndgen=rndgen)
bg_inj.fit(X=exp_, srcs=srcs_, run_dict=run_dict_)

In [ ]:
# Build and fit sig injector
ts = UniformTimeSampler(random_state=None)
sig_inj = SignalFluenceInjector(power_law_flux, time_sampler=ts)
sig_inj.fit(srcs_, MC=MC_)

In [ ]:
# Build and fit llh
mod_opts = None
grb_mod = GRBModel(exp_, MC_, srcs_, model_opts=mod_opts)
llh_opts = None
llh = GRBLLH(llh_model=grb_mod, llh_opts=llh_opts)

In [ ]:
ana = GRBLLHAnalysis(llh, bg_inj, sig_inj)

#### Rate allsky model

In [ ]:
# Redo the allsky rate fit for testing
recs = make_rate_records(T=exp_["timeMJD"], run_dict=run_dict_)
rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                     ignore_zero_runs=True)
mids = 0.5 * (new_bins[:-1] + new_bins[1:])

rate_func = SinusFixedConstRateFunction(p_fix=365.)
allsky_res = rate_func.fit(rate=rates, srcs=srcs_, t=mids, w=1. / stddev)

t0_fix = allsky_res.x[1]
print("Best fit t0 before first event: ",
      t0_fix - exp_["timeMJD"].min(), "days")

rate_func = SinusFixedConstRateFunction(p_fix=365., t0_fix=t0_fix)

#### BG splines and sampling

In [ ]:
# Show build spline models for timedependent injector
x = np.linspace(-1, 1, 100)
bins = bg_inj_opts["sindec_bins"]
mids = 0.5 * (bins[:-1] + bins[1:])
allsky_pars = bg_inj._spl_info["allsky_best_params"]
print("Allsky best params: " + arr2str(allsky_pars))

TEST = True

for n in ["amp", "base"]:
    fig, ax = plt.subplots(1, 1)
    spl = bg_inj._spl_info["param_splines"][n]
    vals = np.copy(bg_inj._spl_info["best_pars_norm"][n])
    err = np.copy(bg_inj._spl_info["best_stddevs_norm"][n])
   
    ax.plot(x, spl(x), color="k")
    ax.plot(mids, vals, color="C7", ls="--")
    ax.errorbar(mids, vals, yerr=err, fmt="o", color="C1")
        
    # Quickly switch smoothing for testing
    if TEST:
        w = 1. / err
        vals_, pts_, w = make_spl_edges(vals=vals, bins=sindec_bins, w=w)
        stop = False
        s_ = bg_inj_opts["spl_s"]
        while not stop:
            spl_ = sci.UnivariateSpline(pts_, vals_, w=w, s=s_)
            norm_ = (allsky_pars[0] if n == "amp" else allsky_pars[-1])

            spl2_ = spl_.derivative(n=2)
            stop = np.all(np.abs(spl2_(x)) < 1.)
            if not stop:
                s_ = s_ * 0.9
                print("Degraded s_. 2nd derivative was ",
                      np.abs(spl2_(x)).max())
        # Renorm to allsky for comparison
        scale_ = norm_ / spl_.integral(-1, 1)
        ax.plot(x, spl_(x) * scale_, color="C0", ls="--")
    
    if n == "amp":
        ax.axhline(0, 0, 1, color="C7", ls="--")
    else:
        ax.set_ylim(0, None)

    ax.set_xlabel("sindec")
    ax.set_ylabel(n)
    ax.set_title("Integral: {:.3f} mHz".format(spl.integral(-1, 1) * 1e3))
    # Show sindec bin borders
    ylim = ax.get_ylim()
    ax.vlines(sindec_bins, ylim[0], ylim[1],
               linestyles=":", colors="C7")
    ylim = ax.set_ylim(ylim)

    plt.show()

In [ ]:
# Compare hist to allsky model
x = np.linspace(-1, 1, 200)
bins = 100
plt.vlines(sindec_bins, 0, 1, linestyles="--", colors="C7", zorder=-1)
plt.hist(np.sin(exp_["dec"]), bins=bins, density=True)
plt.plot(x, bg_inj._spl_info["data_spl"](x))

plt.show()

In [ ]:
# Make multiple trials and concat to compare to spline with large stats
# Use internal debug var to get samples per source to compare to splines
nsrcs = len(bg_inj.srcs)
sam = [list() for _ in range(nsrcs)]
nsamples = 1000
for _ in range(nsamples):
    sami = bg_inj.sample()
    src_idx = bg_inj._sample_idx["src_idx"]
    for j in range(nsrcs):
        sam[j].append(sami[src_idx == j])
        
sam = [np.concatenate(sami) for sami in sam]

In [ ]:
x = np.linspace(-1, 1, 200)
bins = 40
# bins = np.linspace(-1, 1, 17 + 1)
for j, sami in enumerate(sam):
    plt.vlines(sindec_bins, 0, 1, linestyles="--", colors="C7", zorder=-1)
    # Plot allyear sample for comparison
    plt.hist(np.sin(exp_["dec"]), bins=bins, density=True, color="0.75")
    plt.plot(x, bg_inj._spl_info["data_spl"](x), color="C0", ls=":", lw=3)
    # Drawn sample per source. Red hist should approx. follow black spline
    plt.hist(np.sin(sami["dec"]), bins=bins, density=True,
             histtype="step", lw=2.5, color="C3")
    plt.plot(x, bg_inj._spl_info["sin_dec_splines"][j](x), color="k")

    plt.show()

#### MC Injector

In [ ]:
# Testing signal injector
sam = sig_inj.sample(n_samples=10000)
idx = sig_inj._sample_idx

In [ ]:
# Sampled MC hist
for name in ["ra", "dec", "logE", "timeMJD", "sigma"]:
    # Show sampled data-like attributes
    if name != "timeMJD":
        # Compare to full MC pool distribution
        w = sig_inj._MC["ow"] * sig_inj.flux_model(sig_inj._MC["trueE"])
        _ = plt.hist(sig_inj._MC[name], weights=w, density=True, bins=100,
                     alpha=.5)
        _ = plt.hist(sam[name], density=True, bins=100, histtype="step", lw=3)
    if name in ["ra", "dec"]:
        plt.vlines(sig_inj.srcs[name], 0, 1)
        plt.yscale("log", nonposy="clip")
    if name == "timeMJD":
        ts = sig_inj.srcs["t"]
        dt0s, dt1s = sig_inj.srcs["dt0"], sig_inj.srcs["dt1"]
        for j in range(len(ts)):
            plt.title("{}. {:.2f}, {:.2f}, {:.2f}".format(
                name, ts[j], dt0s[j], dt1s[j]))
            mask = (idx["src_idx"] == j)
            trel = (ts[j] - sam[name][mask]) * secinday
            _ = plt.hist(trel, density=False, bins=100,
                         histtype="step", lw=3)
            plt.axvline(0, 0, 1)
            plt.axvline(dt0s[j], 0, 1, ls="--")
            plt.axvline(dt1s[j], 0, 1, ls="--")
            plt.show()
    else:
        plt.title(name)
        plt.show()

#### LLH

In [ ]:
# TODO

#### TEST: Fixate x,y ratio hist edges

Similar to make_spl_edges, but just repeat the outermost values.
We have the full bin range covered with the interpolator, without introducing artifacts.

In [ ]:
# Generate test data
rndgen = np.random.RandomState(3242342)
x = rndgen.uniform(0, 1, size=1000)
y = rndgen.uniform(0, 1, size=1000)

bx = np.linspace(0, 1, 6)
by = np.linspace(0, 1, 11)
bxm, bym = map(lambda b: 0.5 * (b[:-1] + b[1:]), [bx, by])

print(h.shape)
print(len(bx))
print(len(by))

h = plt.hist2d(x, y, bins=[bx, by], normed=True)[0]
plt.colorbar()
plt.show()

In [ ]:
# Repeat outermost values, first in all cols, then the rows
h_ext = np.zeros((len(bxm) + 2, len(bym) + 2), dtype=h.dtype) - 1.
for j, col in enumerate(h):
    h_ext[j+1] = np.concatenate([col[[0]], col, col[[-1]]])
h_ext[0] = h_ext[1]
h_ext[-1] = h_ext[-2]
pts_x = np.concatenate((bx[[0]], bxm, bx[[-1]]))
pts_y = np.concatenate((by[[0]], bym, by[[-1]]))

print(h_ext.shape)
print(len(pts_x))
print(len(pts_y))

# Orignal bins
xx, yy = np.meshgrid(bx, by)
# Original bin mids
xxm, yym = np.meshgrid(bxm, bym)
# New grid points from bin mids
xxg, yyg = np.meshgrid(pts_x, pts_y)

# Plot original hist again
plt.pcolormesh(xx, yy, h.T)
plt.colorbar()
# Plot original bin edges, bin mids and new pts for the interpolator
plt.scatter(xx, yy, marker="o", color="C3", s=50)
plt.scatter(xxm, yym, marker="o", color="w", s=50)
plt.scatter(xxg, yyg, marker=".", color="k")
plt.show()

# Plot rows and cols of the new hist
# Note: The first/last row and col are repeated and are drawn on top of the next
# row/col in the plot
for i, col in enumerate(h_ext):
    plt.plot(pts_y, col, label="{:d}".format(i))
for p in pts_y:
    plt.axvline(p, 0, 1, color="C7", ls="--")
plt.title("Cols")
plt.ylim(0, 2)
plt.legend(loc="upper center", ncol=4)
plt.savefig("/Users/tmenne/Downloads/col_rep")
plt.show()

for i, row in enumerate(h_ext.T):
    plt.plot(pts_x, row, label="{:d}".format(i))
for p in pts_x:
    plt.axvline(p, 0, 1, color="C7", ls="--")
plt.title("Rows")
plt.ylim(0, 2)
plt.legend(loc="upper center", ncol=6)
plt.savefig("/Users/tmenne/Downloads/row_rep")
plt.show()

In [ ]:
# Linearly extrapolate outermost values, first in all cols, then the rows
h_ext = np.zeros((len(bxm) + 2, len(bym) + 2), dtype=h.dtype) - 1.
for j, col in enumerate(h):
    vals, pts_y, _ = make_spl_edges(vals=col, bins=by)
    h_ext[j+1] = vals
for i, row in enumerate(h_ext.T):
    vals, pts_x, _ = make_spl_edges(vals=row[1:-1], bins=bx)
    h_ext[:, i] = vals

print(h_ext.shape)
print(len(pts_x))
print(len(pts_y))

# Orignal bins
xx, yy = np.meshgrid(bx, by)
# Original bin mids
xxm, yym = np.meshgrid(bxm, bym)
# New grid points from bin mids
xxg, yyg = np.meshgrid(pts_x, pts_y)

# Plot original hist again
plt.pcolormesh(xx, yy, h.T)
plt.colorbar()
# Plot original bin edges, bin mids and new pts for the interpolator
plt.scatter(xx, yy, marker="o", color="C3", s=50)
plt.scatter(xxm, yym, marker="o", color="w", s=50)
plt.scatter(xxg, yyg, marker=".", color="k")
plt.show()

# Plot rows and cols of the new hist
for i, col in enumerate(h_ext):
    plt.plot(pts_y, col, label="{:d}".format(i))
for p in pts_y:
    plt.axvline(p, 0, 1, color="C7", ls="--")
plt.title("Cols")
plt.ylim(0, 2)
plt.legend(loc="upper center", ncol=4)
plt.savefig("/Users/tmenne/Downloads/col_lin")
plt.show()

for i, row in enumerate(h_ext.T):
    plt.plot(pts_x, row, label="{:d}".format(i))
for p in pts_x:
    plt.axvline(p, 0, 1, color="C7", ls="--")
plt.title("Rows")
plt.ylim(0, 2)
plt.legend(loc="upper center", ncol=6)
plt.savefig("/Users/tmenne/Downloads/row_lin")
plt.show()

#### TEST: LLH scan instead of hess_inv from fitres

In [ ]:
def plot_llh_scan(bfs, stds, llh, grid):
    """
    Plot the llh scan with errors and contours
    
    Parameters
    ----------
    bfs : array-like, shape (2)
        Best fit result parameters, around which the LLH was scanned.
    stds : array-like, shape (2)
        Approximate standard deviations (symmetric) for each fit parameter,
        obtained using Wilks' theorem on the scanned space.
    llh : array-like, shape (nbins, nbins)
        Scanned LLH values.
    grid : list
        X, Y grid, same shape as ``llh``.
    """
    bf_x, bf_y = bfs
    std_x, std_y = stds
    x, y = grid
    
    # Plot scan
    fig, ax = plt.subplots(1, 1)
    img = ax.pcolormesh(x, y, llh)
    fig.colorbar(img, ax=ax)

    # Plot 1, 2, 3 sigma contours
    vals = np.amin(llh) - scs.chi2.logsf(df=2, x=[1**2, 2**2, 3**2])
    ax.contour(x, y, llh, vals, linestyles=["--", "-.", "--"], colors="w")
    
    # Plot best fit with symmetric errors
    ax.errorbar(bf_x, bf_y, xerr=std_x, yerr=std_y, fmt="o", c="w", capsize=5)   
    
    ax.xlabel = ("amplitude")
    ax.ylabel = ("baseline")

def get_stddev_from_scan(func, args, bfs, rngs, nbins=50):
    """
    Scan the rate_func chi2 fit LLH to get stddevs for the best fit params a, d.
    Using matplotlib contours and averaging to approximately get the variances.
    Note: This is not a true LLH profile scan in both variables.
    
    Parameters
    ----------
    func : callable
        Loss function to be scanned, used to obtain the best fit. Function
        is called as done with a scipy fitter, ``func(x, *args)``.
    args : tuple
        Args passed to the loss function ``func``. For a rate function, this is
        ``(mids, rates, weights)``.
    bfs : array-like, shape (2)
        Best fit result parameters.
    rngs : list
        Parameter ranges to scan: ``[bf[i] - rng[i], bf[i] + rng[i]]``.
    nbins : int, optional
        Number of bins in each dimension to scan. (Default: 100)
        
    Returns
    -------
    stds : array-like, shape (2)
        Approximate standard deviations (symmetric) for each fit parameter,
        obtained using Wilks' theorem on the scanned space.
    llh : array-like, shape (nbins, nbins)
        Scanned LLH values.
    grid : list
        X, Y grid, same shape as ``llh``.
    """
    def _scan_llh(bf_x, rng_x, bf_y, rng_y):
        """ Scan LLH and return contour vertices """
        x_bins = np.linspace(bf_x - rng_x, bf_x + rng_x, nbins)
        y_bins = np.linspace(bf_y - rng_y, bf_y + rng_y, nbins)   
        x, y = np.meshgrid(x_bins, y_bins)
        AA, DD = map(np.ravel, [x, y])
        llh = np.empty_like(AA)
        for i, (ai, di) in enumerate(zip(AA, DD)):
            llh[i] = func((ai, di), *args)
        llh = llh.reshape(x.shape)
        # Get the contour points and average over min, max per parameter
        one_sigma_level = np.amin(llh) - scs.chi2.logsf(df=2, x=[1**2])

        # https://stackoverflow.com/questions/5666056
        cntr = plt.contour(x, y, llh, one_sigma_level)
        plt.close("all")
        paths = [lcol.vertices for lcol in cntr.collections[0].get_paths()]
        # Call undocumented base of plt.contour, to avoid creating a figure.
        # Not working for mpl 2.2.2 any more, because _cntr was deleted.
        # cntr = contour.Cntr(x, y, llh)
        # paths = cntr.trace(level0=one_sigma_level)
        # paths = paths[:len(paths) // 2]  # First half of list has the vertices
        return paths, llh, [x, y]

    def _is_path_closed(paths, rng_x, rng_y):
        """
        We want the contour to be fully contained. Means there is only one path
        and the first and last point are close together.
        Returns ``True`` if contour is closed.
        """
        closed = False
        if len(paths) == 1:
            vertices = paths[0]
            # If no contour is made, only 1 vertex is returned -> invalid
            if len(vertices) > 1:
                max_bin_dist = np.amax([rng_x / float(nbins),
                                        rng_y / float(nbins)])
                closed = np.allclose(vertices[0], vertices[-1],
                                     atol=max_bin_dist, rtol=0.)
        return closed
    
    def _get_stds_from_path(path):
        """ Create symmetric stddevs from the path vertices """
        x, y = path[:, 0], path[:, 1]
        # Average asymmetricities in both direction
        x_min, x_max = np.amin(x), np.amax(x)
        y_min, y_max = np.amin(y), np.amax(y)
        return 0.5 * (x_max - x_min), 0.5 * (y_max - y_min)
           
    # Scan the LLH, adapt scan range if contour is not closed
    bf_x, bf_y = bfs
    rng_x, rng_y = rngs
    closed = False
    while not closed:
        # Default is scaling up, when range is too small
        scalex, scaley = 10., 10.
        # Get contour from scanned LLH space
        paths, llh, grid = _scan_llh(bf_x, rng_x, bf_y, rng_y)
        closed = _is_path_closed(paths, rng_x, rng_y)       
        if closed:
            vertices = paths[0]
            # Estimate scale factors to get contour in optimum resolution
            diffx = np.abs(np.amax(vertices[:, 0]) - np.amin(vertices[:, 0]))
            diffy = np.abs(np.amax(vertices[:, 1]) - np.amin(vertices[:, 1]))
            scalex = diffx / rng_x
            scaley = diffy / rng_y
            # Contour can be closed, but extremely zoomed out in only one param
            if not np.allclose([scalex, scaley], 1., atol=0.5, rtol=0.):
                print("Contour is very distorted in one direction")
                closed = False
            else:
                # Rescan valid contour to use optimal scan resolution
                for i in range(2):
                    std_x, std_y = _get_stds_from_path(vertices)
                    rng_x = std_x * 1.05  # Allow a little padding
                    rng_y = std_y * 1.05
                    paths, llh, grid = _scan_llh(bf_x, rng_x, bf_y, rng_y)
                    # Recheck if path is still valid
                    closed = _is_path_closed(paths, rng_x, rng_y)
        # Must be seperated if, because path can get invalid in rescaling step
        if not closed:
            print("Open or no contour, rescale")
            rng_x *= scalex
            rng_y *= scaley

    vertices = paths[0]
    stds = np.array(_get_stds_from_path(vertices))
    return stds, llh, grid

In [ ]:
sindec = exp_["sinDec"]
t_ = np.linspace(exp_["timeMJD"].min(), exp_["timeMJD"].max(), 200)

allres = []
errs = []
for j, (lo, hi) in enumerate(zip(sindec_bins[:-1], sindec_bins[1:])[:]):
    mask = (sindec >= lo) & (sindec <= hi)

    recs = make_rate_records(T=exp_["timeMJD"][mask], run_dict=run_dict_)
    rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                                ignore_zero_runs=True)
    new_mids = 0.5 * (new_bins[:-1] + new_bins[1:])
    weights = 1. / stddev
    res = rate_func.fit(rate=rates, srcs=srcs_, t=new_mids, w=weights)
    bfs = np.array([res.x[0], res.x[1]])
    allres.append(res)
    
    plt.errorbar(recs["start_mjd"], recs["rate"], yerr=recs["rate_std"],
                 fmt=",", alpha=0.2, color="C0")
    plt.plot(recs["start_mjd"], recs["rate"], marker=".", ls="", color="C0")
    plt.plot(new_bins, np.r_[rates[0], rates], drawstyle="steps-pre", color="k")
    plt.plot(t_, rate_func.fun(t=t_, pars=bfs), color="C3")
    plt.ylim(0, 3. * bfs[1])
    plt.show()
    
    # Empirical seed estimates for amplitude and baseline scan range
    args = (new_mids, rates, weights)
    rngs = np.array([bfs[0], bfs[1] / 10.])
    stds, llh, grid = get_stddev_from_scan(
        func=rate_func._lstsq, args=args, bfs=bfs, rngs=rngs, nbins=20)
    
    plot_llh_scan(bfs, stds, llh, grid)
    plt.show()
    
    errs.append(stds)

errs = np.array(errs)

In [ ]:
# 0 = amp, 1 = base
# Note: The spline is not renormalized here, so there might be differences in
#       scale to the one from the module
for idx in [0, 1]:
    x = np.linspace(-1, 1, 100)
    mids = 0.5 * (sindec_bins[:-1] + sindec_bins[1:])
    norm = np.diff(sindec_bins)

    vals = np.array([res.x[idx] for res in allres]) / norm
    err_ = errs.T[idx] / norm

    # Prepare for spl fit
    w = 1. / err_
    spl, vals, pts, w = fit_spl_to_hist(h=vals, bins=sindec_bins, w=w, s=10)
    
    plt.plot(pts, vals, color="C7", ls="--")
    plt.errorbar(pts, vals, yerr=1. / w, fmt="o", color="C1")
    plt.plot(x, spl(x), color="k")
    plt.title("Integral: {:.3f} mHz".format(spl.integral(-1, 1) * 1e3))

    if idx == 0:
        plt.axhline(0, 0, 1, color="C7", ls="--")
        plt.ylabel("amp")
    else:
        plt.ylim(0, None)
        plt.ylabel("base")
    plt.xlabel("sindec")
 
    plt.show()

#### TEST: Event preselection

In [ ]:
name = "86II_III"

ev_dec = exp[name]["dec"]
ev_ra = exp[name]["ra"]
ev_sigma = exp[name]["sigma"]

src_dec = srcs[name]["dec"][:, None]
src_ra = srcs[name]["ra"][:, None]

nsigma = 1.

# Only mask events in a square with length nsigma * sigma to one of the sources
dec_mask = ((ev_dec > src_dec - ev_sigma * nsigma) &
            (ev_dec < src_dec + ev_sigma * nsigma))
mask = dec_mask
# RA mask needs more thought due to soild angle differences
# ra_mask = ((ev_ra > (src_ra - ev_sigma * nsigma / np.cos(src_dec))) &
#            (ev_ra < (src_ra + ev_sigma * nsigma / np.cos(src_dec))))
# mask = ra_mask & dec_mask

print(np.sum(mask, axis=1))

# Plot events per source
m = (ev_sigma < np.deg2rad(1.))  # Only show well reconstructed events
plt.plot(ev_ra[m], ev_dec[m], color="C7", alpha=.1, marker=".", ls="", ms=1)
for j, m in enumerate(mask[:]):
    m = m & (ev_sigma < np.deg2rad(1.))  # Only show well reconstructed events
    plt.scatter(ev_ra[m], ev_dec[m], s=100 * ev_sigma[m], alpha=.5)
    plt.plot(src_ra[j], src_dec[j], marker="*", ms=5)
    
# Show events contributing to all sources (if any)
m = np.where(np.all(mask, axis=0))
plt.scatter(ev_ra[m], ev_dec[m], s=10 * ev_sigma[m], alpha=1, color="k")

plt.show()

## Multi containers

In [ ]:
# Prepare and fit multi BG injector
hor = np.sin(np.deg2rad(30))
sindec_bins = np.unique(np.concatenate([
                        np.linspace(-1., -hor, 3 + 1),    # south
                        np.linspace(-hor, +hor, 14 + 1),  # horizon
                        np.linspace(+hor, 1., 3 + 1),     # north
                        ]))

bg_injs = {}
bg_inj_opts = {}
for key in exp_.keys():
    print("Fitting BGInj {}".format(key))
    if key == "86II_III":
        n_rate_bins = 24
    else:
        n_rate_bins = 12
    rate_rebins = np.linspace(exp_[key]["timeMJD"].min(),
                              exp_[key]["timeMJD"].max(), n_rate_bins)
    bg_inj_opts[key] = {"sindec_bins": sindec_bins, "rate_rebins": rate_rebins,
                        "spl_s": len(sindec_bins) // 2, "n_scan_bins": 25}

    bg_inj_i = TimeDecDependentBGDataInjector(inj_opts=bg_inj_opts[key],
                                              rndgen=rndgen)
    bg_inj_i.fit(X=exp_[key], srcs=srcs_[key], run_dict=run_dict_[key])
    bg_injs[key] = bg_inj_i
multi_bg_inj = MultiBGDataInjector()
multi_bg_inj.fit(bg_injs)

In [ ]:
# Prepare and fit multi Signal injector
sig_injs = {}
ts = UniformTimeSampler(random_state=None)
for key in exp_.keys():
    print("Fitting SigInj {}".format(key))
    sig_inj_i = SignalFluenceInjector(power_law_flux, time_sampler=ts)
    sig_inj_i.fit(srcs_[key], MC=MC_[key])
    sig_injs[key] = sig_inj_i

multi_sig_inj = MultiSignalFluenceInjector()
multi_sig_inj.fit(sig_injs)

In [ ]:
# Prepare and fit multi LLH
llhs = {}
for key in exp_names.keys():
    print("Preparing LLH {}".format(key))
    pdf_i = GRBPDF(exp_[key], MC_[key], srcs_[key])
    llh_args = None
    llh_i = GRBLLH(model_pdf=pdf_i, llh_args=llh_args)
    llhs[key] = llh_i
multi_llh = MultiGRBLLH()
multi_llh.fit(llhs=llhs)

In [ ]:
multi_ana = GRBLLHAnalysis(multi_llh, multi_bg_inj, multi_sig_inj)

#### BG splines and sampling

Test if multi sampler samples correctly from each single injector.
In the BG case this just collects all single samples.

In [ ]:
n_samples = 1000
multi_bg_sam_ = [multi_bg_inj.sample() for i in range(n_samples)]
multi_bg_sam = {}
for key in multi_bg_inj.names:
    multi_bg_sam[key] = np.concatenate([sam_i[key] for sam_i in multi_bg_sam_])
nsam = map(len, multi_bg_sam.values())
for i, key in enumerate(multi_bg_inj.names):
    print("{:8s} : {}".format(key, nsam[i]))
    
nbins = 100
for name in ["ra", "dec", "logE", "sigma"]:
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    for i, key in enumerate(sorted(multi_bg_inj.names)):
        ax[i].hist(multi_bg_sam[key][name], bins=nbins)
        ax[i].set_title(key)
    fig.tight_layout()
    plt.show()

Show build spline models for timedependent injector

In [ ]:
TEST = False
x = np.linspace(-1, 1, 100)
for key, bg_inj in multi_bg_inj._injs.items():
    print("Plotting for sample '{}'".format(key))

    bins = bg_inj_opts[key]["sindec_bins"]
    mids = 0.5 * (bins[:-1] + bins[1:])
    print("Allsky best params: " + arr2str(
        bg_inj._spl_info["allsky_best_params"]))

    for n in ["amp", "base"]:
        fig, ax = plt.subplots(1, 1)
        spl = bg_inj._spl_info["param_splines"][n]
        vals = np.copy(bg_inj._spl_info["best_pars_norm"][n])
        err = np.copy(bg_inj._spl_info["best_stddevs_norm"][n])

        ax.plot(x, spl(x), color="k")
        ax.plot(mids, vals, color="C7", ls="--")
        ax.errorbar(mids, vals, yerr=err, fmt="o", color="C1")

        # Quickly switch smoothing for testing
        if TEST:
            w = 1. / err
            vals_, pts_, w = make_spl_edges(vals=vals, bins=sindec_bins, w=w)
            stop = False
            s_ = bg_inj_opts[key]["spl_s"]
            while not stop:
                spl_ = sci.UnivariateSpline(pts_, vals_, w=w, s=s_)
                norm_ = (bg_inj._spl_info["allsky_best_params"][0] if n == "amp"
                         else bg_inj._spl_info["allsky_best_params"][-1])

                spl2_ = spl_.derivative(n=2)
                stop = np.all(np.abs(spl2_(x)) < 1.)
                if not stop:
                    s_ = s_ * 0.9
                    print("Degraded s_. 2nd derivative was ",
                          np.abs(spl2_(x)).max())
            # Renorm to allsky for comparison
            scale_ = norm_ / spl_.integral(-1, 1)
            ax.plot(x, spl_(x) * scale_, color="C0", ls="--")

        if n == "amp":
            ax.axhline(0, 0, 1, color="C7", ls="--")
        else:
            ax.set_ylim(0, None)

        ax.set_xlabel("sindec")
        ax.set_ylabel(n)
        ax.set_title("Integral: {:.3f} mHz".format(spl.integral(-1, 1) * 1e3))
        # Show sindec bin borders
        ylim = ax.get_ylim()
        ax.vlines(sindec_bins, ylim[0], ylim[1],
                   linestyles=":", colors="C7")
        ylim = ax.set_ylim(ylim)

        plt.show()

Plot rates vs rate model for each sample

In [ ]:
for key, bg_inj in sorted(multi_bg_inj._injs.items())[:]:
    recs = make_rate_records(run_dict=run_dict_[key], T=exp_[key]["timeMJD"])   
    rebin = rebin_rate_rec(rate_rec=recs, bins=bg_inj_opts[key]["rate_rebins"],
                           ignore_zero_runs=True)
    rates, bins, rate_std, _ = rebin

    mids = 0.5 * (recs["start_mjd"] + recs["stop_mjd"])
    diff = recs["stop_mjd"] - recs["start_mjd"]
    t = np.linspace(bins[0], bins[-1], 200)
    
    # Plot it
    plt.errorbar(mids, recs["rate"], xerr=diff, yerr=recs["rate_std"], fmt=",",
                 color="C0", alpha=0.5)
    plt.plot(bins, np.r_[rates[0], rates], drawstyle="steps-pre", color="k")
    plt.plot(t, bg_inj._spl_info["allsky_rate_func"].bf_fun(t),
             color="C3", lw=2.5)
        
    plt.ylim(0, 0.01)
    plt.xlabel("time in MJD days")
    plt.ylabel("Rate in mHz")
    plt.title("Allsky rate model for sample {}".format(key))
    plt.show()

Plot rate model for each sindec bin per sample

In [ ]:
for key, bg_inj in sorted(multi_bg_inj._injs.items())[:]:
    T = exp_[key]["timeMJD"]
    sindec = np.sin(exp_[key]["dec"])
    sd_bins = bg_inj_opts[key]["sindec_bins"]
    rate_fun = bg_inj._spl_info["allsky_rate_func"]

    nplots, nrows, ncols = 20, 4, 5
    assert ncols * nrows == nplots
    assert nplots == len(sindec_bins) - 1
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(24, 13.5),
                           sharex=True, sharey=True)
    
    for i, (lo, hi) in enumerate(zip(sd_bins[:-1], sd_bins[1:])):
        mask = (sindec >= lo) & (sindec < hi)              
        recs = make_rate_records(run_dict=run_dict_[key], T=T[mask])
        rebin = rebin_rate_rec(
            rate_rec=recs, bins=bg_inj_opts[key]["rate_rebins"],
            ignore_zero_runs=True)
        rates, bins, rate_std, _ = rebin
        
        mids = 0.5 * (recs["start_mjd"] + recs["stop_mjd"])
        diff = recs["stop_mjd"] - recs["start_mjd"]
        t = np.linspace(bins[0], bins[-1], 200)
        
        amp, base = bg_inj._spl_info["best_pars"][i]
        pars = (amp, bg_inj._spl_info["allsky_best_params"][1], base)
        lab = "{:.2f} <= sindec < {:.2f}".format(lo, hi)
        
        # Plot it
        row, col = idx2rowcol(i, ncols=ncols)
        ax_ = ax[row, col]
        ax_.errorbar(mids, recs["rate"], xerr=diff, yerr=recs["rate_std"],
                     fmt=",", color="C0", alpha=0.5, label=lab)
        ax_.plot(bins, np.r_[rates[0], rates], drawstyle="steps-pre", color="k")
        ax_.plot(t, rate_fun.fun(t, pars), color="C3", lw=2.5)

        ax_.set_ylim(0, 0.001)
        ax_.legend(loc="upper right")
    ax[0, 0].text(s="Sample: '{}'".format(key), x=bins[0], y=0.0009,
                  bbox={"facecolor": "w", "alpha": 0.5}, fontsize=12)
    fig.tight_layout()
    plt.show()

In [ ]:
# Compare hist to allsky model
x = np.linspace(-1, 1, 200)
bins = 100
for key, bg_inj in sorted(multi_bg_inj._injs.items())[:]:
    plt.vlines(bg_inj_opts[key]["sindec_bins"], 0, 1, linestyles="--",
               colors="C7", zorder=-1)
    plt.hist(np.sin(exp_[key]["dec"]), bins=bins, density=True)
    plt.plot(x, bg_inj._spl_info["data_spl"](x))

    plt.show()

#### MC injector

Test splitting of signal samples

In [ ]:
nsamples = 10000
multi_sig_sam = multi_sig_inj.sample(n_samples=nsamples)
print(sum(map(len, (multi_sig_sam.values()))))
print(dict_map(lambda key, sam: len(sam), multi_sig_sam))
print(dict_map(lambda key, wts: wts * nsamples,
               multi_sig_inj._distribute_weights))

dict_map(lambda key, val: np.sum(val) / multi_sig_inj.flux2mu(1.),
         multi_sig_inj.flux2mu(1., per_source=True))

In [ ]:
for key, sami in multi_sig_sam.items():
    # Sampled MC hist
    _inj = multi_sig_inj.injs[key]
    _idx = multi_sig_inj.injs[key]._sample_idx
    for name in ["ra", "dec", "logE", "timeMJD", "sigma"]:
        # Show sampled data-like attributes
        if name != "timeMJD":
            # Compare to full MC pool distribution
            w = _inj._MC["ow"] * _inj.flux_model(_inj._MC["trueE"])
            _ = plt.hist(_inj._MC[name], weights=w, density=True, bins=100,
                         alpha=.5)
            _ = plt.hist(sami[name], density=True, bins=100,
                         histtype="step", lw=3)
        if name in ["ra", "dec"]:
            plt.vlines(_inj.srcs[name], 0, 1)
            plt.yscale("log", nonposy="clip")
        if name == "timeMJD":
            ts = _inj.srcs["t"]
            dt0s, dt1s = _inj.srcs["dt0"], _inj.srcs["dt1"]
            for j in range(len(ts)):
                plt.title("{}. {:.2f}, {:.2f}, {:.2f}".format(
                    name, ts[j], dt0s[j], dt1s[j]))
                mask = (_idx["src_idx"] == j)
                trel = (ts[j] - sami[name][mask]) * secinday
                _ = plt.hist(trel, density=False, bins=100,
                             histtype="step", lw=3)
                plt.axvline(0, 0, 1)
                plt.axvline(dt0s[j], 0, 1, ls="--")
                plt.axvline(dt1s[j], 0, 1, ls="--")
                plt.show()
        else:
            plt.title("{}: {}".format(key, name))
            plt.show()